# Key Concepts

欢迎来到 `braincell` 的世界！

本节简要介绍了 `braincell` 框架的一些关键概念。

`braincell` 是一个专为神经元建模设计的高性能计算框架，基于[JAX](https://github.com/jax-ml/jax) 构建。它为神经科学研究者、计算神经科学家和类脑计算工程师提供了一套完整的工具链，用于构建、模拟和优化从离子到多尺度神经网络的电生理精确模型，并且整合了现代硬件加速、自动微分等先进特性。以下教程将详细介绍其层次架构和核心功能，帮助你快速理解 `braincell` 的构建逻辑。

```python
import jax.numpy as jnp
import braincell
```

## 层次架构

在神经科学建模中，我们通常将模型按照结构复杂度与功能层次，划分为以下几个主要层级：

- `System`
- `Network`
- `Cell`
- `Channel`
- `Ion`

从上到下，结构逐渐简化，功能逐渐单一。

`braincell` 用于进行精确的神经元动力学建模，专注于其中的 `Cell`、`Channel` 和 `Ion` 三个层级。
- `Cell`：单个神经元的建模单位，用于整合多个通道并产生膜电位变化。
- `Channel`：膜上的离子通道，控制特定离子的通透性，决定电流流动。
- `Ion`：最基本的带电离子，如 Na⁺、K⁺、Ca²⁺ 等，驱动膜电位变化。

三者大体的结构如下图：

![此图正在绘制](../../)

通过这张图不难看出，`Cell`、`Channel` 和 `Ion`在结构上有着非常紧密的联系。从生物学上来看，这三个层级构成了神经元电活动的基本机制：
- 神经元细胞`Cell`由细胞膜包围，膜上分布着各种离子通道`Channel`。
- 每个离子通道专门控制一种或几种离子`Ion`的进出。
- 离子的流动会引起膜内外电位的变化，从而驱动神经元的活动。

这些层级构成了神经元从微观机制到宏观动力学的完整建模路径。

## 核心功能

`braincell` 的主要功能包括以下几个部分：
- 神经元建模：提供基于 HH 模型的单房室和多房室神经元结构。
- 离子通道建模：支持通过模块化组件构建电生理精确的离子通道模型。
- 离子建模：支持通过模块化组件构建的离子模型。
- 微分方程求解：支持基于[JAX](https://github.com/jax-ml/jax) 的高性能微分方程求解器。

接下来，我们将逐项深入探讨这些功能的使用方法和优化策略，帮助你从完整的神经元到精细的离子，全面掌握 ``braincell`` 的建模能力。

我们将依次介绍：
- `Cell`：如何对神经元进行建模，并组合其中的离子通道。
- `Channel`：如何对离子通道进行建模。
- `Ion`：如何对离子进行建模。

通过逐步构建，我们将最终实现一个完整的神经元模型，并观察其电生理行为。

此处需要进行补充：`Cell` 不是实际存在的类，而是为了方便讲解归纳的一个层级，实际上 `HHTypedNeuron` 是神经元建模的基类。但其他两个 `braincell` 关注的层次，即 `Ion` 和 `Channel` ，都是真实存在的类。这一点在后续建模中我们会进行讲解。

